In [1]:
# To be run after German-Data-Prep
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import balanced_accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
#from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_German, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned German dataset
German_df = pd.read_csv('./input/German-Cleaned.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'Gender'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'CreditStatus'
fav_label           = 1 # Good Credit status
unfav_label         = 0 # Bad Credit Status
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create the traoning and test splits
X = German_df.drop(lab,axis=1)
y = German_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
German_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
German_train_bld, German_test_bld = German_bld.split([0.8], shuffle=True)

In [7]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
German_train_bld.features = min_max_scaler.fit_transform(German_train_bld.features)
German_test_bld.features  = min_max_scaler.fit_transform(German_test_bld.features)

In [8]:
# Convert the train and test datasets to dataframes
German_train_df, d = German_train_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
German_test_df,  d = German_test_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [9]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
X_train = German_train_df.drop(lab,axis=1)
y_train = German_train_df[lab]
X_test  = German_test_df.drop(lab,axis=1)
y_test  = German_test_df[lab]
BiasedLogModel = LogisticRegression(random_state=101)
BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
BiasedLogModel.fit(X_train, y_train) 
BiasedRfcModel.fit(X_train, y_train)
BiasedLogPredictions = BiasedLogModel.predict(X_test)
BiasedRfcPredictions = BiasedRfcModel.predict(X_test)
print(f"Biased Logistic regression validation accuracy: {BiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {BiasedRfcModel.score(X_test, y_test)}")
print('')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, BiasedLogPredictions)}")
print(f"Biased Random forest balanced accuracy        : {balanced_accuracy_score(y_test, BiasedRfcPredictions)}")

Biased Logistic regression validation accuracy: 0.78
Biased Random Forest       validation accuracy: 0.75

Biased Logistic regression balanced accuracy  : 0.6331096196868009
Biased Random forest balanced accuracy        : 0.5484932227924727


In [10]:
print('Training Before - CreditStatus value counts:')
print(German_train_df.CreditStatus.value_counts())
print('Training Before - Gender value counts:')
print(German_train_df.Gender.value_counts())

Training Before - CreditStatus value counts:
1.0    551
0.0    249
Name: CreditStatus, dtype: int64
Training Before - Gender value counts:
1.0    552
0.0    248
Name: Gender, dtype: int64


In [11]:
print('Test Before - CreditStatus value counts:')
print(German_test_df.CreditStatus.value_counts())
print('Test Before - Gender value counts:')
print(German_test_df.Gender.value_counts())

Test Before - CreditStatus value counts:
1.0    149
0.0     51
Name: CreditStatus, dtype: int64
Test Before - Gender value counts:
1.0    138
0.0     62
Name: Gender, dtype: int64


In [12]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(German_train_bld, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_test_bld = BinaryLabelDatasetMetric(German_test_bld, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

In [13]:
display(Markdown("#### Orig training dataset"))
print('Number of instances           :', metric_train_bld.num_instances())
print('Base Rate                     :', metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
display(Markdown("#### Orig test dataset"))
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

#### Orig training dataset

Number of instances           : 800.0
Base Rate                     : 0.68875
Consistency                   : [0.69475]
Disparate Impact              : 0.9028143515470703
Mean Difference               : -0.06901589527816743
Statistical Parity Difference : -0.06901589527816743


#### Orig test dataset

Number of instances           : 200.0
Base Rate                     : 0.745
Consistency                   : [0.705]
Disparate Impact              : 0.8736810370817003
Mean Difference               : -0.09794296400187008
Statistical Parity Difference : -0.09794296400187008


In [14]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [15]:
biased_model.fit(German_train_bld)

W0831 16:45:30.512720 11284 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0831 16:45:30.517286 11284 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 16:45:30.537343 11284 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0831 16:45:32.945299 11284 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.703604
epoch 1; iter: 0; batch classifier loss: 0.691012
epoch 2; iter: 0; batch classifier loss: 0.676081
epoch 3; iter: 0; batch classifier loss: 0.670612
epoch 4; iter: 0; batch classifier loss: 0.686446
epoch 5; iter: 0; batch classifier loss: 0.653131
epoch 6; iter: 0; batch classifier loss: 0.615117
epoch 7; iter: 0; batch classifier loss: 0.632963
epoch 8; iter: 0; batch classifier loss: 0.619563
epoch 9; iter: 0; batch classifier loss: 0.572984
epoch 10; iter: 0; batch classifier loss: 0.559056
epoch 11; iter: 0; batch classifier loss: 0.592551
epoch 12; iter: 0; batch classifier loss: 0.619934
epoch 13; iter: 0; batch classifier loss: 0.577404
epoch 14; iter: 0; batch classifier loss: 0.602236
epoch 15; iter: 0; batch classifier loss: 0.566416
epoch 16; iter: 0; batch classifier loss: 0.509396
epoch 17; iter: 0; batch classifier loss: 0.563039
epoch 18; iter: 0; batch classifier loss: 0.584925
epoch 19; iter: 0; batch classifier loss:

In [16]:
# Apply the plain model to test data
German_biased_train = biased_model.predict(German_train_bld)
German_biased_test  = biased_model.predict(German_test_bld)
# And convert them to dataframes
German_train_df, d = German_biased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
German_test_df, d  = German_biased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [17]:
# Test whether it is possible to predict the Sensitive Variable from the biased dataset
X_se_train = German_train_df.drop(protected_attr,axis=1)
y_se_train = German_train_df[protected_attr]
X_se_test  = German_test_df.drop(protected_attr,axis=1)
y_se_test  = German_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.765
Biased Random Forest       validation accuracy: 0.7

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.6431743805516597
Biased Random Forest       balanced accuracy  : 0.5250116877045349


In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_biased_train = BinaryLabelDatasetMetric(German_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_biased_test = BinaryLabelDatasetMetric(German_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_biased_train.num_instances())
print('Base Rate                     :', metric_biased_train.base_rate())
print('Consistency                   :', metric_biased_train.consistency())
print('Disparate Impact              :', metric_biased_train.disparate_impact())
print('Mean Difference               :', metric_biased_train.mean_difference())
print('Statistical Parity Difference :', metric_biased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_biased_test.num_instances())
print('Base Rate                     :', metric_biased_test.base_rate())
print('Consistency                   :', metric_biased_test.consistency())
print('Disparate Impact              :', metric_biased_test.disparate_impact())
print('Mean Difference               :', metric_biased_test.mean_difference())
print('Statistical Parity Difference :', metric_biased_test.statistical_parity_difference()) 

#### Plain model - without debiasing - dataset metrics

#### Biased training dataset

Number of instances           : 800.0
Base Rate                     : 0.84
Consistency                   : [0.883]
Disparate Impact              : 0.9297672519395671
Mean Difference               : -0.06030855539971958
Statistical Parity Difference : -0.06030855539971958


#### Biased test dataset

Number of instances           : 200.0
Base Rate                     : 0.805
Consistency                   : [0.838]
Disparate Impact              : 0.8110989611809732
Mean Difference               : -0.16152407667134172
Statistical Parity Difference : -0.16152407667134172


In [19]:
# Create the Classification metric for train and test
classified_metric_biased_test = ClassificationMetric(German_train_bld, German_biased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_biased_train = ClassificationMetric(German_test_bld, German_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [20]:
display(Markdown("#### Plain model - without debiasing - classification metrics"))

display(Markdown("#### Biased training dataset - CLassification Metrics"))
TPR_train = classified_metric_biased_train.true_positive_rate()
TNR_train = classified_metric_biased_train.true_negative_rate()
bal_acc_biased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_biased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_train)
print('Disparate impact                =', classified_metric_biased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_train.theil_index())

display(Markdown("#### Biased test dataset - CLassification Metrics"))
TPR_test = classified_metric_biased_test.true_positive_rate()
TNR_test = classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_biased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_test)
print('Disparate impact                =', classified_metric_biased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - classification metrics

#### Biased training dataset - CLassification Metrics

Classification accuracy         = 0.79
Balanced classification accuracy= 0.6849585471772601
Disparate impact                = 0.8110989611809732
Equal opportunity difference    = -0.1250556297285269
Average odds difference         = -0.12785039550942476
Theil_index                     = 0.11828744918846304


#### Biased test dataset - CLassification Metrics

Classification accuracy         = 0.74625
Balanced classification accuracy= 0.6374937135110315
Disparate impact                = 0.9297672519395671
Equal opportunity difference    = -0.010332434860736761
Average odds difference         = -0.05678138596969423
Theil_index                     = 0.10299499923175418


In [21]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [22]:
unbiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [23]:
unbiased_model.fit(German_train_bld)

epoch 0; iter: 0; batch classifier loss: 0.690254; batch adversarial loss: 0.809726
epoch 1; iter: 0; batch classifier loss: 0.650387; batch adversarial loss: 0.775556
epoch 2; iter: 0; batch classifier loss: 0.624248; batch adversarial loss: 0.794394
epoch 3; iter: 0; batch classifier loss: 0.609331; batch adversarial loss: 0.843962
epoch 4; iter: 0; batch classifier loss: 0.655286; batch adversarial loss: 0.822060
epoch 5; iter: 0; batch classifier loss: 0.598702; batch adversarial loss: 0.809299
epoch 6; iter: 0; batch classifier loss: 0.592609; batch adversarial loss: 0.840333
epoch 7; iter: 0; batch classifier loss: 0.599786; batch adversarial loss: 0.816233
epoch 8; iter: 0; batch classifier loss: 0.592673; batch adversarial loss: 0.853873
epoch 9; iter: 0; batch classifier loss: 0.590208; batch adversarial loss: 0.834423
epoch 10; iter: 0; batch classifier loss: 0.634373; batch adversarial loss: 0.841582
epoch 11; iter: 0; batch classifier loss: 0.535046; batch adversarial loss:

In [24]:
German_unbiased_train = unbiased_model.predict(German_train_bld)
German_unbiased_test  = unbiased_model.predict(German_test_bld)
# And convert these to dataframes..
German_train_df, d = German_unbiased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
German_test_df, d  = German_unbiased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [25]:
# Metrics for the dataset from plain model (after unbiasing)
display(Markdown("#### Plain model - after unbiasing - dataset metrics"))
metric_unbiased_train = BinaryLabelDatasetMetric(German_unbiased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_unbiased_test = BinaryLabelDatasetMetric(German_unbiased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_unbiased_train.num_instances())
print('Base Rate                     :', metric_unbiased_train.base_rate())
print('Consistency                   :', metric_unbiased_train.consistency())
print('Disparate Impact              :', metric_unbiased_train.disparate_impact())
print('Mean Difference               :', metric_unbiased_train.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_unbiased_test.num_instances())
print('Base Rate                     :', metric_unbiased_test.base_rate())
print('Consistency                   :', metric_unbiased_test.consistency())
print('Disparate Impact              :', metric_unbiased_test.disparate_impact())
print('Mean Difference               :', metric_unbiased_test.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_test.statistical_parity_difference()) 

#### Plain model - after unbiasing - dataset metrics

#### Biased training dataset

Number of instances           : 800.0
Base Rate                     : 0.79
Consistency                   : [0.89025]
Disparate Impact              : 0.44854654728320864
Mean Difference               : -0.525479195885928
Statistical Parity Difference : -0.525479195885928


#### Biased test dataset

Number of instances           : 200.0
Base Rate                     : 0.81
Consistency                   : [0.886]
Disparate Impact              : 0.4650938854116514
Mean Difference               : -0.5194015895278168
Statistical Parity Difference : -0.5194015895278168


In [26]:
# Create the Classification metric for train and test
classified_metric_unbiased_train = ClassificationMetric(German_train_bld, German_unbiased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_unbiased_test = ClassificationMetric(German_test_bld, German_unbiased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [27]:
display(Markdown("#### Plain model - after debiasing - classification metrics"))
display(Markdown("#### Unbiased training dataset - CLassification Metrics"))
TPR_train = classified_metric_unbiased_train.true_positive_rate()
TNR_train = classified_metric_unbiased_train.true_negative_rate()
bal_acc_unbiased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_unbiased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_train)
print('Disparate impact                =', classified_metric_unbiased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_train.theil_index())

display(Markdown("#### Unbiased test dataset - CLassification Metrics"))
TPR_test = classified_metric_unbiased_test.true_positive_rate()
TNR_test = classified_metric_unbiased_test.true_negative_rate()
bal_acc_unbiased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_unbiased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_test)
print('Disparate impact                =', classified_metric_unbiased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_test.theil_index())

#### Plain model - after debiasing - classification metrics

#### Unbiased training dataset - CLassification Metrics

Classification accuracy         = 0.71875
Balanced classification accuracy= 0.6274353311613059
Disparate impact                = 0.44854654728320864
Equal opportunity difference    = -0.4174688743421897
Average odds difference         = -0.5557147742497466
Theil_index                     = 0.1443066979307753


#### Unbiased test dataset - CLassification Metrics

Classification accuracy         = 0.745
Balanced classification accuracy= 0.6225161205421765
Disparate impact                = 0.4650938854116514
Equal opportunity difference    = -0.38607031597685804
Average odds difference         = -0.5857770934722999
Theil_index                     = 0.14529477621812567


In [28]:
# Determine the after-transformation model accuracy for Logistic Regression and Random Forest Classifiers
X_train = German_train_df.drop(lab,axis=1)
y_train = German_train_df[lab]
X_test  = German_test_df.drop(lab,axis=1)
y_test  = German_test_df[lab]

UnbiasedLogModel = LogisticRegression(random_state=101)
UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_train, y_train) 
UnbiasedRfcModel.fit(X_train, y_train) 
UnbiasedLog_pred = UnbiasedLogModel.predict(X_test)
UnbiasedRfc_pred = UnbiasedRfcModel.predict(X_test)

# Now get Logistic Regression and Random Forest Clasisfication performancce of unbiased data
print('Model Accuracy for Log Reg and RFC after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {UnbiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {UnbiasedRfcModel.score(X_test, y_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedRfc_pred)}")

Model Accuracy for Log Reg and RFC after bias transformation:
Biased Logistic regression validation accuracy: 0.93
Biased Random Forest       validation accuracy: 0.92

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.8762183235867447
Biased Random Forest       balanced accuracy  : 0.7995451591942819


In [29]:
# Finally test whether it is possible to predict the Sensitive Variable from the transformed dataset
X_se_train = German_train_df.drop(protected_attr,axis=1)
y_se_train = German_train_df[protected_attr]
X_se_test  = German_test_df.drop(protected_attr,axis=1)
y_se_test  = German_test_df[protected_attr]

Se_unbiasedLogModel = LogisticRegression(random_state=101)
Se_unbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_unbiasedLogModel.fit(X_se_train, y_se_train) 
Se_unbiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_unbiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_unbiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_unbiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_unbiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.85
Biased Random Forest       validation accuracy: 0.835

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.7669471715755025
Biased Random Forest       balanced accuracy  : 0.751636278634876


In [30]:
print('Predicted Training After - CreditStatus value counts:')
print(German_train_df.CreditStatus.value_counts())
print('Predicted Training After - Gender value counts:')
print(German_train_df.Gender.value_counts())

Predicted Training After - CreditStatus value counts:
1.0    632
0.0    168
Name: CreditStatus, dtype: int64
Predicted Training After - Gender value counts:
1.0    552
0.0    248
Name: Gender, dtype: int64


In [31]:
print('Predicted Test After - CreditStatus value counts:')
print(German_test_df.CreditStatus.value_counts())
print('Predicted Test After - Gender value counts:')
print(German_test_df.Gender.value_counts())

Predicted Test After - CreditStatus value counts:
1.0    162
0.0     38
Name: CreditStatus, dtype: int64
Predicted Test After - Gender value counts:
1.0    138
0.0     62
Name: Gender, dtype: int64
